In [108]:
from lyricsgenius import Genius
from bs4 import BeautifulSoup
import requests
import pandas as pd
from itertools import islice
import os

# Get top 50 rap songs between 2018 and 2022 from billboard website

In [109]:
def get_top50_from_url(year):
    #example = url_start + '2018' + url_end
    url_start = 'https://www.billboard.com/charts/year-end/'
    url_end = '/hot-rap-songs/'

    url = url_start + str(year) + url_end
    response = requests.get(url)

    soup = BeautifulSoup(response.text, 'html.parser')
    results = soup.find_all('div', {'class': 'o-chart-results-list-row-container'})

    data = []

    for i in range(len(results)):
        result = results[i]
        title = result.find('h3').text.strip()
        spans = result.find_all('span')
        artist = spans[1].text.strip()
        data.append({'position': i+1, 'title': title, 'artist': artist, 'year': year})

    df = pd.DataFrame(data)
    return df


In [110]:
# Get top 50 songs for each year
df = pd.DataFrame(columns=['position', 'title', 'artist', 'year'])
for i in range(2018,2023):
    df = df.append(get_top50_from_url(i), ignore_index=True)

df.head()

C:\Users\Surface\AppData\Local\Temp\ipykernel_632\3513559878.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(get_top50_from_url(i), ignore_index=True)
C:\Users\Surface\AppData\Local\Temp\ipykernel_632\3513559878.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(get_top50_from_url(i), ignore_index=True)
C:\Users\Surface\AppData\Local\Temp\ipykernel_632\3513559878.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(get_top50_from_url(i), ignore_index=True)
C:\Users\Surface\AppData\Local\Temp\ipykernel_632\3513559878.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(get_to

,position,title,artist,year
0,1,God's Plan,Drake,2018
1,2,I Like It,"Cardi B, Bad Bunny & J Balvin",2018
2,3,In My Feelings,Drake,2018
3,4,Psycho,Post Malone Featuring Ty Dolla $ign,2018
4,5,Nice For What,Drake,2018


In [111]:
# write to csv
df.to_csv('top50.csv', index=False)

# Use lyricsmaster to get lyrics for each song from Genius.com API

In [123]:
def remove_special_chars(file_name):
    # Define special characters to remove
    special_chars = "!@#$%^*()[]{};:/<>?\|`~-=_+’"

    # Remove special characters from file name
    new_file_name = ''.join(c for c in file_name if c not in special_chars)

    return new_file_name

In [ ]:
i = 0
def scrape_lyrics():
    # Authenticate with Genius API
    genius = Genius('uWk7jiPtIjLiomyfQS1Y6QvB_Aw3EWQr3ODfulXHBKZU1kBWFBzNwJMQ0rbi3Ig6')
    # create the subdirectory /songs/ if it does not exist
    if not os.path.exists('songs'):
        os.makedirs('songs')
    # read the index.txt file to get the index of the last song that was written to a json file
    with open('index.txt', 'r') as f:
        i = int(f.read())
    # iterate through each row in the dataframe starting at index i
    for row,column in islice(df.iterrows(), i, None):
        song = genius.search_song(column['title'], column['artist'])
        # write the song object to a json file named by the song title and artist name in the folder songs
        file_name = 'song' + str(i) + '.json'
        # modify the file name to remove all special characters that are not allowed in a file name
        file_name = remove_special_chars(file_name)
        song.save_lyrics(file_name, overwrite=True)
        # move the created json into the /songs/ folder
        os.rename(os.getcwd() + '\\' + file_name, os.getcwd() + '\\' + 'songs' + '\\' + file_name)
        # write a txt file with the content i and the name index.txt
        i += 1
        with open('index.txt', 'w') as f:
            f.write(str(i))
        df.loc[row, 'lyrics'] = song.lyrics

while i < len(df):
    try:
        scrape_lyrics()
    except requests.exceptions.Timeout:
        print('Request timed out')
    except requests.exceptions.RequestException as e:
        print('Error:', e)


Searching for "Stir Fry" by Migos...
Done.
Wrote song20.json.
Searching for "Nonstop" by Drake...
Done.
Wrote song21.json.
Searching for "I Fall Apart" by Post Malone...
Done.
Wrote song22.json.
Searching for "Gummo" by 6ix9ine...
Done.
Wrote song23.json.
Searching for "Ric Flair Drip" by Offset & Metro Boomin...
Done.
Wrote song24.json.
Searching for "This Is America" by Childish Gambino...
Done.
Wrote song25.json.
Searching for "Plug Walk" by Rich The Kid...
Done.
Wrote song26.json.
Searching for "Be Careful" by Cardi B...
Done.
Wrote song27.json.
Searching for "Love." by Kendrick Lamar Featuring Zacari...
Done.
Wrote song28.json.
Searching for "Bartier Cardi" by Cardi B Featuring 21 Savage...
Done.
Wrote song29.json.
Searching for "Big Bank" by YG Featuring 2 Chainz, Big Sean & Nicki Minaj...
Done.
Wrote song30.json.
Searching for "Plain Jane" by A$AP Ferg Featuring Nicki Minaj...
Done.
Wrote song31.json.
Searching for "I Get The Bag" by Gucci Mane Featuring Migos...
Done.
Wrote son